In [5]:
# установка plotly
!pip install plotly

     |████████████████████████████████| 25.3MB 371kB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
# импортируем необхожимые библиотеки 
import pandas as pd
import plotly.graph_objects as go

In [7]:
# создаем dataframe

df = pd.read_csv('GAZP.csv', sep = ';')
df

,DATE,OPEN,HIGH,LOW,CLOSE
0,13.12.2018,154.95,156.49,153.25,156.49
1,14.12.2018,155.96,155.98,153.51,155.40
2,17.12.2018,155.16,156.41,152.93,153.53
3,18.12.2018,152.40,152.42,150.05,151.89
4,19.12.2018,151.10,153.79,150.71,152.98
...,...,...,...,...,...
752,07.12.2021,339.25,346.00,333.43,344.96
753,08.12.2021,346.00,348.47,336.53,340.50
754,09.12.2021,341.45,344.20,335.72,336.60
755,10.12.2021,337.02,338.31,329.98,332.00


In [8]:
# построение графика на основе ohlc
fig = go.Figure(data=go.Ohlc(x=df['DATE'],
                    open=df['OPEN'],
                    high=df['HIGH'],
                    low=df['LOW'],
                    close=df['CLOSE']))
fig.update_layout(
    title='GAZPROM ohlc visualization',
    yaxis_title='GAZP Stock')

fig.show()

In [9]:
# построение скользящих средних
short = 30
long = 90 
df['SMA'] = df['CLOSE'].rolling(short).mean()
df['LMA'] = df['CLOSE'].rolling(long).mean()
# удаляем значения до появления LMA
df = df.drop(df.index[0:long-1])

In [10]:
# строим график скользящие и ohlc
from plotly.subplots import make_subplots


fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Ohlc(x=df['DATE'], open=df['OPEN'], high=df['HIGH'], low=df['LOW'], close=df['CLOSE'],
                             yaxis='y1', name='OHLC'))
fig.add_trace(go.Scatter(x=df['DATE'], y=df['SMA'], name='Short',
                        line=dict(color='royalblue',width=2)))
fig.add_trace(go.Scatter(x=df['DATE'], y=df['LMA'], name='Long',
                        line=dict(color='orange',width=2)))

In [11]:
# определяем места пересечения скользящих и потенциальные места выхода на рынок, находим первую точку выхода на рынок.
slow_ma = df['LMA'].to_list()
fast_ma = df['SMA'].to_list()
status = slow_ma[0] < fast_ma[0]
if status == True:
    flag = ("покупка","продажа")
else:
    flag = ("продажа", "покупка")

change_position = []
for ind, value in enumerate(slow_ma):
    if status == (value < fast_ma[ind]):
        continue
    else:

        if len(change_position) % 2 == 0 :
            direction = flag[0]
        else: direction = flag[1]
        change_position.append((ind, direction))
        status = not status
if len(change_position) == 0:
    print("Нет мест пересечения скользящих для покупки")
elif len(change_position) == 1 and change_position[0][1] == "продажа":
    print("Нет мест пересечения скользящих для покупки")
    
if change_position[0][1] == "продажа":
    del change_position[0]
    
change_position

[(113, 'покупка'),
 (135, 'продажа'),
 (200, 'покупка'),
 (289, 'продажа'),
 (320, 'покупка'),
 (410, 'продажа')]

В блоке кода выше получили, все точки пересечения скользящих. Эти точки будут полезны для определения точных дат выхода на рынок. Данные из change_position по покупкам будут совпадать, а по продажам в зависимости от того, как сильно будут дешеветь акции. И того как часто будет срабатывать stop loss

In [12]:
# описываем класс инвестора. Даем ему свойства покупать и продавать. По дефолту 
# инвестор имеет 0 акций и 1000р актива.
# В history складываем историю всех выходов на рынок.
class Investor():
    def __init__(self, money=1000, stock=0):
        self.money = money
        self.stock = stock
        self.history = []
          # этот список можно использовать для подведения итогов, в случае если позиция открыта, а рассматриваемый период закончен. 
        self.after_sale = [] 

    def to_buy(self, price, ind):
        self.stock = self.money / price
        self.history.append((ind, "покупка", self.money))
        self.money = 0

    def to_sell(self, price, ind):
        self.money = self.stock * price
        self.stock = 0
        self.history.append((ind, "продажа", self.money))


In [16]:
somebody = Investor(money=1000)

# Совершаем первую покупку. Заводим промежуточные переменные для рассчетов
pricies = df['OPEN'].to_list()
first_purchase_ind = change_position[0][0] + 1
purchase_price = pricies[first_purchase_ind]
somebody.to_buy(purchase_price, first_purchase_ind)
position_open = True

# заводим стоп лосс в stop_loss_procent. Можно менять в зависимости от задачи
stop_loss_procent = 20
ind_for_sale = [i[0] + 1 for i in change_position if i[1] == 'продажа']
ind_for_purchase = [i[0] + 1 for i in change_position if i[1] == 'покупка']

# Проверяем в цикле на стоп лосс и совершаем сделки.
for ind, price in enumerate(pricies[first_purchase_ind + 1:]):
    cust_ind = ind + first_purchase_ind + 1
    if position_open == True and cust_ind in ind_for_sale:
        somebody.to_sell(pricies[cust_ind], cust_ind)
        position_open = False
    if position_open == True and price <= (1-stop_loss_procent/100) * purchase_price:
        somebody.to_sell(pricies[cust_ind], cust_ind)
        position_open = False
        print('stop loss work')
    if position_open == False and cust_ind in ind_for_purchase:
        purchase_price = pricies[cust_ind]
        somebody.to_buy(purchase_price, cust_ind)
        position_open = True

print(somebody.money)

stop loss work
911.1362968262878


В блоке кода выше реализована модель ведения торгов. Торги велись объектом класса Investor(), инициализированным с 1000р. На имеющихся данных получены следующие результаты. При stop_loss = 5 процентам, в ходе торгов получаем прибыль в 29р, при stop_loss = 10 процентам, получаем убыток в 43р. А при stop_loss = 20 процентам, убыток в 89 рублей. 

In [17]:
# добавляем в исходный датафрейм данные о сделках. Заводим структуры данных, которые
# в дальнейшем будут использоваться для графического отображения выходов на рынок. 
buy_lst = []
sale_lst = []
buy_price = []
sale_price = []
start_b = 0
start_s = 0
for i in somebody.history:
    if i[1] == "покупка":
        buy_lst = buy_lst + [None for i in range(start_b, i[0])]
        buy_lst.append(i[2])
        buy_price = buy_price + [None for i in range(start_b, i[0])]
        buy_price.append(slow_ma[len(buy_lst)])
        start_b = len(buy_lst)
    if i[1] == "продажа":
        sale_lst = sale_lst + [None for i in range(start_s, i[0])]
        sale_lst.append(i[2])
        sale_price = sale_price + [None for i in range(start_s, i[0])]
        sale_price.append(fast_ma[len(sale_lst)])
        start_s = len(sale_lst)
buy_lst = buy_lst + [None for i in range(start_b, len(pricies))]
sale_lst = sale_lst + [None for i in range(start_s, len(pricies))]
sale_price = sale_price + [None for i in range(start_s, len(pricies))]
buy_price = buy_price + [None for i in range(start_b, len(pricies))]
df["Buy"] = buy_lst
df["Sale"] = sale_lst

In [19]:
# отражаем на графике места выхода на рынок.
from plotly.validators.scatter.marker import SymbolValidator


fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Ohlc(x=df['DATE'], open=df['OPEN'], high=df['HIGH'], low=df['LOW'], close=df['CLOSE'],
                             yaxis='y1', name='OHLC'))
fig.add_trace(go.Scatter(x=df['DATE'], y=df['LMA'], name='Short',
                        line=dict(color='royalblue',width=2)))
fig.add_trace(go.Scatter(x=df['DATE'], y=df['SMA'], name='Long',
                        line=dict(color='orange',width=2)))

raw = SymbolValidator().values

fig.add_trace(go.Scatter(
    mode = 'markers',
    marker_symbol = raw[60],
    x=df['DATE'],
    y=buy_price,
    marker=dict(color="green", size=12),
    name="Buy",
))

fig.add_trace(go.Scatter(
    mode="markers",
    marker_symbol=raw[80],
    x=df['DATE'],
    y= sale_price,
    marker=dict(color="red", size=12),
    name="Sale",
))